In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from category_encoders import TargetEncoder

In [2]:
# Load the dataset
parquet_file = "imperial_dataset_20240523.parquet"
df = pd.read_parquet(parquet_file, engine='pyarrow')

# Display basic information about the dataset
print("Basic Information:")
print(df.info())


Basic Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290802 entries, 0 to 290801
Data columns (total 69 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   ID_QUOTE                          290802 non-null  int64  
 1   DAYSTOINCEPTION                   290799 non-null  float64
 2   MARITALSTATUS                     290799 non-null  object 
 3   NOOFCHILDREN                      290799 non-null  float64
 4   HOMEOWNERIND                      290799 non-null  object 
 5   VEHICLE_DATEMANUFACTURED          290799 non-null  object 
 6   VEHICLE_LEFTORRIGHTHANDDRIVE      290799 non-null  object 
 7   VEHICLE_DATEFIRSTREGD             290799 non-null  object 
 8   VEHICLE_PURCHASEDATE              290799 non-null  object 
 9   DRIVER1_NONMOTORINGCONVICTIONIND  290799 non-null  object 
 10  DRIVER2_NONMOTORINGCONVICTIONIND  159597 non-null  object 
 11  DRIVER3_NONMOTORINGCONVICTIONIND 

In [3]:
df = df.drop('ID_QUOTE', axis=1)

In [4]:
# Drop the data where have a row of missing value, or have missing value on targeted variable TOP5PREMIUM.
df = df.dropna(subset=['DAYSTOINCEPTION', 'TOP5PREMIUM'])

In [5]:
df[df['TOP5PREMIUM'].isna()]

,DAYSTOINCEPTION,MARITALSTATUS,NOOFCHILDREN,HOMEOWNERIND,VEHICLE_DATEMANUFACTURED,VEHICLE_LEFTORRIGHTHANDDRIVE,VEHICLE_DATEFIRSTREGD,VEHICLE_PURCHASEDATE,DRIVER1_NONMOTORINGCONVICTIONIND,DRIVER2_NONMOTORINGCONVICTIONIND,...,DRIVER1_CLAIMSCOUNT_RECENT,DRIVER2_CLAIMSCOUNT_RECENT,DRIVER3_CLAIMSCOUNT_RECENT,DRIVER4_CLAIMSCOUNT_RECENT,DRIVER1_CLAIMSCOUNT_LONGER,DRIVER2_CLAIMSCOUNT_LONGER,DRIVER3_CLAIMSCOUNT_LONGER,DRIVER4_CLAIMSCOUNT_LONGER,UNDISCLOSED_CLAIMS,TOP5PREMIUM


In [6]:
print("Basic Information:")
print(df.info())

Basic Information:
<class 'pandas.core.frame.DataFrame'>
Index: 106074 entries, 0 to 279410
Data columns (total 68 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   DAYSTOINCEPTION                   106074 non-null  float64
 1   MARITALSTATUS                     106074 non-null  object 
 2   NOOFCHILDREN                      106074 non-null  float64
 3   HOMEOWNERIND                      106074 non-null  object 
 4   VEHICLE_DATEMANUFACTURED          106074 non-null  object 
 5   VEHICLE_LEFTORRIGHTHANDDRIVE      106074 non-null  object 
 6   VEHICLE_DATEFIRSTREGD             106074 non-null  object 
 7   VEHICLE_PURCHASEDATE              106074 non-null  object 
 8   DRIVER1_NONMOTORINGCONVICTIONIND  106074 non-null  object 
 9   DRIVER2_NONMOTORINGCONVICTIONIND  63206 non-null   object 
 10  DRIVER3_NONMOTORINGCONVICTIONIND  14038 non-null   object 
 11  DRIVER4_NONMOTORINGCONVICTIONIND  1585

In [7]:
null_rows_count = df[
    df['DRIVER4_AGE'].notnull() & 
    df['DRIVER4_OTHERVEHICLEOWNEDIND'].notnull() & 
    df['DRIVER4_CREDITSCORE'].notnull()
].shape[0]
print(f"Number of rows where DRIVER4_AGE, DRIVER4_OTHERVEHICLEOWNEDIND, and DRIVER4_CREDITSCORE are all not null: {null_rows_count}")

null_rows_count = df[
    df['DRIVER3_AGE'].notnull() & 
    df['DRIVER3_OTHERVEHICLEOWNEDIND'].notnull() & 
    df['DRIVER3_CREDITSCORE'].notnull()
].shape[0]
print(f"Number of rows where DRIVER3_AGE, DRIVER3_OTHERVEHICLEOWNEDIND, and DRIVER3_CREDITSCORE are all not null: {null_rows_count}")

null_rows_count = df[
    df['DRIVER2_AGE'].notnull() & 
    df['DRIVER2_OTHERVEHICLEOWNEDIND'].notnull() & 
    df['DRIVER2_CREDITSCORE'].notnull()
].shape[0]
print(f"Number of rows where DRIVER2_AGE, DRIVER2_OTHERVEHICLEOWNEDIND, and DRIVER2_CREDITSCORE are all not null: {null_rows_count}")

Number of rows where DRIVER4_AGE, DRIVER4_OTHERVEHICLEOWNEDIND, and DRIVER4_CREDITSCORE are all not null: 1568
Number of rows where DRIVER3_AGE, DRIVER3_OTHERVEHICLEOWNEDIND, and DRIVER3_CREDITSCORE are all not null: 13899
Number of rows where DRIVER2_AGE, DRIVER2_OTHERVEHICLEOWNEDIND, and DRIVER2_CREDITSCORE are all not null: 62642


In [8]:
# From the outputs above, we found some of the additional drivers (Driver 2, 3 and 4) have unknown credit scores.
# We found the columns DRIVER_NRCONVICTIONS_RECENT, DRIVER_NRCONVICTIONS_LONGER, DRIVER_CLAIMSCOUNT_RECENT, and DRIVER_CLAIMSCOUNT_LONGER have wrong inputs as for some data points there are no additional drivers.

In [9]:
# Define the condition where DRIVER2_AGE, DRIVER2_OTHERVEHICLEOWNEDIND, and DRIVER2_CREDITSCORE are all null
condition_driver2 = (
    df['DRIVER2_AGE'].isnull() & 
    df['DRIVER2_OTHERVEHICLEOWNEDIND'].isnull() & 
    df['DRIVER2_CREDITSCORE'].isnull()
)

# Set the specified columns to null where the condition is true for DRIVER2
columns_to_nullify_driver2 = [
    'DRIVER2_NRCONVICTIONS_RECENT', 
    'DRIVER2_NRCONVICTIONS_LONGER', 
    'DRIVER2_CLAIMSCOUNT_RECENT', 
    'DRIVER2_CLAIMSCOUNT_LONGER'
]

df.loc[condition_driver2, columns_to_nullify_driver2] = np.nan

In [10]:
# Define the condition where DRIVER3_AGE, DRIVER3_OTHERVEHICLEOWNEDIND, and DRIVER3_CREDITSCORE are all null
condition_driver3 = (
    df['DRIVER3_AGE'].isnull() & 
    df['DRIVER3_OTHERVEHICLEOWNEDIND'].isnull() & 
    df['DRIVER3_CREDITSCORE'].isnull()
)

# Set the specified columns to null where the condition is true for DRIVER3
columns_to_nullify_driver3 = [
    'DRIVER3_NRCONVICTIONS_RECENT', 
    'DRIVER3_NRCONVICTIONS_LONGER', 
    'DRIVER3_CLAIMSCOUNT_RECENT', 
    'DRIVER3_CLAIMSCOUNT_LONGER'
]

df.loc[condition_driver3, columns_to_nullify_driver3] = np.nan

In [11]:
# Define the condition where DRIVER4_AGE, DRIVER4_OTHERVEHICLEOWNEDIND, and DRIVER4_CREDITSCORE are all null
condition_driver4 = (
    df['DRIVER4_AGE'].isnull() & 
    df['DRIVER4_OTHERVEHICLEOWNEDIND'].isnull() & 
    df['DRIVER4_CREDITSCORE'].isnull()
)

# Set the specified columns to null where the condition is true
columns_to_nullify_driver4 = [
    'DRIVER4_NRCONVICTIONS_RECENT', 
    'DRIVER4_NRCONVICTIONS_LONGER', 
    'DRIVER4_CLAIMSCOUNT_RECENT', 
    'DRIVER4_CLAIMSCOUNT_LONGER'
]

df.loc[condition_driver4, columns_to_nullify_driver4] = np.nan

In [12]:
print("Basic Information:")
print(df.info())

Basic Information:
<class 'pandas.core.frame.DataFrame'>
Index: 106074 entries, 0 to 279410
Data columns (total 68 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   DAYSTOINCEPTION                   106074 non-null  float64
 1   MARITALSTATUS                     106074 non-null  object 
 2   NOOFCHILDREN                      106074 non-null  float64
 3   HOMEOWNERIND                      106074 non-null  object 
 4   VEHICLE_DATEMANUFACTURED          106074 non-null  object 
 5   VEHICLE_LEFTORRIGHTHANDDRIVE      106074 non-null  object 
 6   VEHICLE_DATEFIRSTREGD             106074 non-null  object 
 7   VEHICLE_PURCHASEDATE              106074 non-null  object 
 8   DRIVER1_NONMOTORINGCONVICTIONIND  106074 non-null  object 
 9   DRIVER2_NONMOTORINGCONVICTIONIND  63206 non-null   object 
 10  DRIVER3_NONMOTORINGCONVICTIONIND  14038 non-null   object 
 11  DRIVER4_NONMOTORINGCONVICTIONIND  1585

In [13]:
# Function to Display the Top 5 Rows
def display_top_5_rows(dataframe, block_size=10):
    num_columns = dataframe.shape[1]
    for start in range(0, num_columns, block_size):
        end = start + block_size
        print(f"\nColumns {start + 1} to {min(end, num_columns)}:")
        print(dataframe.iloc[:, start:end].head())

# Function to Display Missing Values
def display_missing_values(dataframe, block_size=10):
    num_columns = dataframe.shape[1]
    for start in range(0, num_columns, block_size):
        end = start + block_size
        print(f"\nColumns {start + 1} to {min(end, num_columns)}:")
        missing_values = dataframe.iloc[:, start:end].isnull().sum()
        print(missing_values)

# Function to Display Unique Values
def display_unique_values(dataframe, block_size=10):
    num_columns = dataframe.shape[1]
    for start in range(0, num_columns, block_size):
        end = start + block_size
        print(f"\nColumns {start + 1} to {min(end, num_columns)}:")
        unique_values = dataframe.iloc[:, start:end].nunique()
        print(unique_values)


In [14]:
# Display the top 5 rows in blocks of 10 columns
display_top_5_rows(df, block_size=10)


Columns 1 to 10:
   DAYSTOINCEPTION MARITALSTATUS  NOOFCHILDREN HOMEOWNERIND  \
0              2.0             S           1.0            Y   
1             21.0             S           0.0            N   
2             29.0             S           0.0            N   
3             10.0             M           1.0            Y   
4              1.0             S           0.0            Y   

  VEHICLE_DATEMANUFACTURED VEHICLE_LEFTORRIGHTHANDDRIVE VEHICLE_DATEFIRSTREGD  \
0               01/01/2015                            R            01/01/2015   
1               01/01/2014                            R            01/01/2014   
2               01/01/2015                            R            01/01/2015   
3               01/01/2009                            R            01/01/2009   
4               01/01/2022                            R            01/01/2022   

  VEHICLE_PURCHASEDATE DRIVER1_NONMOTORINGCONVICTIONIND  \
0           08/11/2023                                N  

In [15]:
# Display missing values in blocks of 10 columns
display_missing_values(df, block_size=10)


Columns 1 to 10:
DAYSTOINCEPTION                         0
MARITALSTATUS                           0
NOOFCHILDREN                            0
HOMEOWNERIND                            0
VEHICLE_DATEMANUFACTURED                0
VEHICLE_LEFTORRIGHTHANDDRIVE            0
VEHICLE_DATEFIRSTREGD                   0
VEHICLE_PURCHASEDATE                    0
DRIVER1_NONMOTORINGCONVICTIONIND        0
DRIVER2_NONMOTORINGCONVICTIONIND    42868
dtype: int64

Columns 11 to 20:
DRIVER3_NONMOTORINGCONVICTIONIND     92036
DRIVER4_NONMOTORINGCONVICTIONIND    104489
DRIVER1_AGE                              0
DRIVER2_AGE                          42868
DRIVER3_AGE                          92036
DRIVER4_AGE                         104489
DRIVER1_LICENCETYPE                      0
DRIVER2_LICENCETYPE                  42868
DRIVER3_LICENCETYPE                  92036
DRIVER4_LICENCETYPE                 104489
dtype: int64

Columns 21 to 30:
DRIVER1_NOOFYEARSUKRESIDENCY          0
DRIVER2_NOOFYEARSUKRESIDENCY

In [16]:
# Display unique values in blocks of 10 columns
display_unique_values(df, block_size=10)


Columns 1 to 10:
DAYSTOINCEPTION                      30
MARITALSTATUS                         7
NOOFCHILDREN                          2
HOMEOWNERIND                          2
VEHICLE_DATEMANUFACTURED             44
VEHICLE_LEFTORRIGHTHANDDRIVE          2
VEHICLE_DATEFIRSTREGD                44
VEHICLE_PURCHASEDATE                274
DRIVER1_NONMOTORINGCONVICTIONIND      1
DRIVER2_NONMOTORINGCONVICTIONIND      1
dtype: int64

Columns 11 to 20:
DRIVER3_NONMOTORINGCONVICTIONIND     1
DRIVER4_NONMOTORINGCONVICTIONIND     1
DRIVER1_AGE                         66
DRIVER2_AGE                         66
DRIVER3_AGE                         66
DRIVER4_AGE                         65
DRIVER1_LICENCETYPE                  5
DRIVER2_LICENCETYPE                  5
DRIVER3_LICENCETYPE                  5
DRIVER4_LICENCETYPE                  5
dtype: int64

Columns 21 to 30:
DRIVER1_NOOFYEARSUKRESIDENCY       83
DRIVER2_NOOFYEARSUKRESIDENCY       83
DRIVER3_NOOFYEARSUKRESIDENCY       83
DRIVER4_NOOFYE

In [17]:
columns_to_delete = [
    'DRIVER1_NONMOTORINGCONVICTIONIND',
    'DRIVER2_NONMOTORINGCONVICTIONIND',
    'DRIVER3_NONMOTORINGCONVICTIONIND',
    'DRIVER4_NONMOTORINGCONVICTIONIND'
]
df.drop(columns=columns_to_delete, inplace=True)

In [18]:
# 1. Handling Missing Values (Feature Engineering)

# As we have too much missing values in some of the columns, so we want to check if these columns (features) are important before we handle the missing values.
# The other way to handle these missing values is feature engineering, that we may want to present the information in another form.

# We create new features: Number of drivers, Average, Maximum, Minimum 

In [19]:
# Count the number of drivers based on non-missing age values
df['NUMBER_OF_DRIVERS'] = df[['DRIVER1_AGE', 'DRIVER2_AGE', 'DRIVER3_AGE', 'DRIVER4_AGE']].notna().sum(axis=1)

# Create additional features for drivers' ages
df['MIN_DRIVER_AGE'] = df[['DRIVER1_AGE', 'DRIVER2_AGE', 'DRIVER3_AGE', 'DRIVER4_AGE']].min(axis=1)
df['MAX_DRIVER_AGE'] = df[['DRIVER1_AGE', 'DRIVER2_AGE', 'DRIVER3_AGE', 'DRIVER4_AGE']].max(axis=1)
df['AVERAGE_DRIVER_AGE'] = df[['DRIVER1_AGE', 'DRIVER2_AGE', 'DRIVER3_AGE', 'DRIVER4_AGE']].mean(axis=1)

# Create additional features for UK residency
df['MIN_YEARS_UK_RESIDENCY'] = df[['DRIVER1_NOOFYEARSUKRESIDENCY', 'DRIVER2_NOOFYEARSUKRESIDENCY', 'DRIVER3_NOOFYEARSUKRESIDENCY', 'DRIVER4_NOOFYEARSUKRESIDENCY']].min(axis=1)
df['MAX_YEARS_UK_RESIDENCY'] = df[['DRIVER1_NOOFYEARSUKRESIDENCY', 'DRIVER2_NOOFYEARSUKRESIDENCY', 'DRIVER3_NOOFYEARSUKRESIDENCY', 'DRIVER4_NOOFYEARSUKRESIDENCY']].max(axis=1)
df['AVERAGE_YEARS_UK_RESIDENCY'] = df[['DRIVER1_NOOFYEARSUKRESIDENCY', 'DRIVER2_NOOFYEARSUKRESIDENCY', 'DRIVER3_NOOFYEARSUKRESIDENCY', 'DRIVER4_NOOFYEARSUKRESIDENCY']].mean(axis=1)

# Create additional features for credit scores
df['MIN_CREDIT_SCORE'] = df[['DRIVER1_CREDITSCORE', 'DRIVER2_CREDITSCORE', 'DRIVER3_CREDITSCORE', 'DRIVER4_CREDITSCORE']].min(axis=1)
df['MAX_CREDIT_SCORE'] = df[['DRIVER1_CREDITSCORE', 'DRIVER2_CREDITSCORE', 'DRIVER3_CREDITSCORE', 'DRIVER4_CREDITSCORE']].max(axis=1)
df['AVERAGE_CREDIT_SCORE'] = df[['DRIVER1_CREDITSCORE', 'DRIVER2_CREDITSCORE', 'DRIVER3_CREDITSCORE', 'DRIVER4_CREDITSCORE']].mean(axis=1)

# Create indicator features for DRIVER_NRCONVICTIONS_RECENT
df['INDICATOR_NRCONVICTIONS_RECENT'] = df[['DRIVER1_NRCONVICTIONS_RECENT', 'DRIVER2_NRCONVICTIONS_RECENT', 'DRIVER3_NRCONVICTIONS_RECENT', 'DRIVER4_NRCONVICTIONS_RECENT']].gt(0).any(axis=1).map({True: 'Y', False: 'N'})

# Create indicator features for DRIVER_CLAIMSCOUNT_RECENT
df['INDICATOR_CLAIMSCOUNT_RECENT'] = df[['DRIVER1_CLAIMSCOUNT_RECENT', 'DRIVER2_CLAIMSCOUNT_RECENT', 'DRIVER3_CLAIMSCOUNT_RECENT', 'DRIVER4_CLAIMSCOUNT_RECENT']].gt(0).any(axis=1).map({True: 'Y', False: 'N'})

# Create additional features for DRIVER_NRCONVICTIONS_LONGER
df['MIN_NRCONVICTIONS_LONGER'] = df[['DRIVER1_NRCONVICTIONS_LONGER', 'DRIVER2_NRCONVICTIONS_LONGER', 'DRIVER3_NRCONVICTIONS_LONGER', 'DRIVER4_NRCONVICTIONS_LONGER']].min(axis=1)
df['MAX_NRCONVICTIONS_LONGER'] = df[['DRIVER1_NRCONVICTIONS_LONGER', 'DRIVER2_NRCONVICTIONS_LONGER', 'DRIVER3_NRCONVICTIONS_LONGER', 'DRIVER4_NRCONVICTIONS_LONGER']].max(axis=1)
df['AVERAGE_NRCONVICTIONS_LONGER'] = df[['DRIVER1_NRCONVICTIONS_LONGER', 'DRIVER2_NRCONVICTIONS_LONGER', 'DRIVER3_NRCONVICTIONS_LONGER', 'DRIVER4_NRCONVICTIONS_LONGER']].mean(axis=1)

# Create additional features for DRIVER_CLAIMSCOUNT_LONGER
df['MIN_CLAIMSCOUNT_LONGER'] = df[['DRIVER1_CLAIMSCOUNT_LONGER', 'DRIVER2_CLAIMSCOUNT_LONGER', 'DRIVER3_CLAIMSCOUNT_LONGER', 'DRIVER4_CLAIMSCOUNT_LONGER']].min(axis=1)
df['MAX_CLAIMSCOUNT_LONGER'] = df[['DRIVER1_CLAIMSCOUNT_LONGER', 'DRIVER2_CLAIMSCOUNT_LONGER', 'DRIVER3_CLAIMSCOUNT_LONGER', 'DRIVER4_CLAIMSCOUNT_LONGER']].max(axis=1)
df['AVERAGE_CLAIMSCOUNT_LONGER'] = df[['DRIVER1_CLAIMSCOUNT_LONGER', 'DRIVER2_CLAIMSCOUNT_LONGER', 'DRIVER3_CLAIMSCOUNT_LONGER', 'DRIVER4_CLAIMSCOUNT_LONGER']].mean(axis=1)

# Delete the original columns used for calculations
columns_to_delete = [
    'DRIVER2_AGE', 'DRIVER3_AGE', 'DRIVER4_AGE',
    'DRIVER2_NOOFYEARSUKRESIDENCY', 'DRIVER3_NOOFYEARSUKRESIDENCY', 'DRIVER4_NOOFYEARSUKRESIDENCY',
    'DRIVER2_CREDITSCORE', 'DRIVER3_CREDITSCORE', 'DRIVER4_CREDITSCORE',
    'DRIVER1_NRCONVICTIONS_RECENT', 'DRIVER2_NRCONVICTIONS_RECENT', 'DRIVER3_NRCONVICTIONS_RECENT', 'DRIVER4_NRCONVICTIONS_RECENT',
    'DRIVER2_NRCONVICTIONS_LONGER', 'DRIVER3_NRCONVICTIONS_LONGER', 'DRIVER4_NRCONVICTIONS_LONGER',
    'DRIVER1_CLAIMSCOUNT_RECENT', 'DRIVER2_CLAIMSCOUNT_RECENT', 'DRIVER3_CLAIMSCOUNT_RECENT', 'DRIVER4_CLAIMSCOUNT_RECENT',
    'DRIVER2_CLAIMSCOUNT_LONGER', 'DRIVER3_CLAIMSCOUNT_LONGER', 'DRIVER4_CLAIMSCOUNT_LONGER'
]
df.drop(columns=columns_to_delete, inplace=True)

# Check the newly created features
display_top_5_rows(df, block_size=10)


Columns 1 to 10:
   DAYSTOINCEPTION MARITALSTATUS  NOOFCHILDREN HOMEOWNERIND  \
0              2.0             S           1.0            Y   
1             21.0             S           0.0            N   
2             29.0             S           0.0            N   
3             10.0             M           1.0            Y   
4              1.0             S           0.0            Y   

  VEHICLE_DATEMANUFACTURED VEHICLE_LEFTORRIGHTHANDDRIVE VEHICLE_DATEFIRSTREGD  \
0               01/01/2015                            R            01/01/2015   
1               01/01/2014                            R            01/01/2014   
2               01/01/2015                            R            01/01/2015   
3               01/01/2009                            R            01/01/2009   
4               01/01/2022                            R            01/01/2022   

  VEHICLE_PURCHASEDATE  DRIVER1_AGE DRIVER1_LICENCETYPE  
0           08/11/2023         21.0                   F  


In [20]:
# Some adjustments after feature engineering
# Convert the column NUMBER_OF_DRIVERS from int64 to float64
df['NUMBER_OF_DRIVERS'] = df['NUMBER_OF_DRIVERS'].astype('float64')

# Move column TOP5PREMIUM to the last position of dataframe
df = df[[col for col in df if col != 'TOP5PREMIUM'] + ['TOP5PREMIUM']]

# Verify the new order of columns
print(df.columns)


Index(['DAYSTOINCEPTION', 'MARITALSTATUS', 'NOOFCHILDREN', 'HOMEOWNERIND',
       'VEHICLE_DATEMANUFACTURED', 'VEHICLE_LEFTORRIGHTHANDDRIVE',
       'VEHICLE_DATEFIRSTREGD', 'VEHICLE_PURCHASEDATE', 'DRIVER1_AGE',
       'DRIVER1_LICENCETYPE', 'DRIVER2_LICENCETYPE', 'DRIVER3_LICENCETYPE',
       'DRIVER4_LICENCETYPE', 'DRIVER1_NOOFYEARSUKRESIDENCY',
       'DRIVER1_OTHERVEHICLEOWNEDIND', 'DRIVER2_OTHERVEHICLEOWNEDIND',
       'DRIVER3_OTHERVEHICLEOWNEDIND', 'DRIVER4_OTHERVEHICLEOWNEDIND',
       'POSTCODESECTOR', 'NOOFVEHICLESAVAILABLETOFAMILY', 'MONTHLYPAYMENTSIND',
       'YEARS_NOCLAIMSDISCOUNT', 'VEHICLE_VALUE', 'VEHICLE_ANNUALMILEAGE',
       'VEHICLE_USE_TYPE', 'VEHICLEOVERNIGHTLOCATION', 'VEHICLE_OWNERSHIP',
       'VEHICLE_MODIFIEDIND', 'VEHICLE_AGE', 'VEHICLE_BODYTYPE',
       'VEHICLE_FUELTYPE', 'VEHICLE_TRANSMISSIONTYPE', 'VEHICLE_NOOFSEATS',
       'DRIVER1_CREDITSCORE', 'VEHICLE_MODEL', 'VEHICLE_MODELNAME',
       'VEHICLE_CUBICCAPACITY', 'DRIVER1_NRCONVICTIONS_LONGER',
   

In [21]:
print("Basic Information:")
print(df.info())

Basic Information:
<class 'pandas.core.frame.DataFrame'>
Index: 106074 entries, 0 to 279410
Data columns (total 59 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   DAYSTOINCEPTION                 106074 non-null  float64
 1   MARITALSTATUS                   106074 non-null  object 
 2   NOOFCHILDREN                    106074 non-null  float64
 3   HOMEOWNERIND                    106074 non-null  object 
 4   VEHICLE_DATEMANUFACTURED        106074 non-null  object 
 5   VEHICLE_LEFTORRIGHTHANDDRIVE    106074 non-null  object 
 6   VEHICLE_DATEFIRSTREGD           106074 non-null  object 
 7   VEHICLE_PURCHASEDATE            106074 non-null  object 
 8   DRIVER1_AGE                     106074 non-null  float64
 9   DRIVER1_LICENCETYPE             106074 non-null  object 
 10  DRIVER2_LICENCETYPE             63206 non-null   object 
 11  DRIVER3_LICENCETYPE             14038 non-null   object 
 12  DR

In [22]:
# Categorise different columns for different processings and manipulations. 

In [23]:
numerical_columns = []
date_columns = []
categorical_columns = []
text_columns = []

# Function to categorize columns based on their descriptions and data types
for column in df.columns:
    if column == 'NOOFCHILDREN':
        categorical_columns.append(column)
    elif pd.api.types.is_numeric_dtype(df[column]):
        numerical_columns.append(column)
    elif 'date' in column.lower() or 'day' in column.lower():
        date_columns.append(column)
    elif pd.api.types.is_string_dtype(df[column]):
        unique_values_count = df[column].nunique()
        if unique_values_count > 50:  # Assuming columns with more than 50 unique values are text columns
            text_columns.append(column)
        else:
            categorical_columns.append(column)
    else:
        # Check if column might be a date even if it's not a string type
        try:
            pd.to_datetime(df[column])
            date_columns.append(column)
        except (ValueError, TypeError):
            pass  # If conversion fails, do nothing

# Any columns not categorized should be rechecked and forced into a category
all_categorized_columns = set(numerical_columns + date_columns + categorical_columns + text_columns)
uncategorized_columns = set(df.columns) - all_categorized_columns

for column in uncategorized_columns:
    if pd.api.types.is_numeric_dtype(df[column]):
        numerical_columns.append(column)
    elif pd.api.types.is_string_dtype(df[column]):
        unique_values_count = df[column].nunique()
        if unique_values_count > 50:
            text_columns.append(column)
        else:
            categorical_columns.append(column)
    else:
        # By default, treat any remaining uncategorized columns as categorical or text based on unique values count
        unique_values_count = df[column].nunique()
        if unique_values_count > 50:
            text_columns.append(column)
        else:
            categorical_columns.append(column)

# Convert date columns to datetime if not already
for column in date_columns:
    df[column] = pd.to_datetime(df[column], errors='coerce', dayfirst=True)

# Display categorized columns
print("Numerical Columns:")
print(numerical_columns)

print("\nDate Columns:")
print(date_columns)

print("\nCategorical Columns:")
print(categorical_columns)

print("\nText Columns:")
print(text_columns)

# Identify and print any columns that were not categorized
uncategorized_columns = set(df.columns) - set(numerical_columns) - set(date_columns) - set(categorical_columns) - set(text_columns)
print("\nUncategorized Columns:")
print(list(uncategorized_columns))

Numerical Columns:
['DAYSTOINCEPTION', 'DRIVER1_AGE', 'DRIVER1_NOOFYEARSUKRESIDENCY', 'NOOFVEHICLESAVAILABLETOFAMILY', 'YEARS_NOCLAIMSDISCOUNT', 'VEHICLE_VALUE', 'VEHICLE_ANNUALMILEAGE', 'VEHICLE_AGE', 'VEHICLE_NOOFSEATS', 'DRIVER1_CREDITSCORE', 'VEHICLE_CUBICCAPACITY', 'DRIVER1_NRCONVICTIONS_LONGER', 'DRIVER1_CLAIMSCOUNT_LONGER', 'UNDISCLOSED_CLAIMS', 'NUMBER_OF_DRIVERS', 'MIN_DRIVER_AGE', 'MAX_DRIVER_AGE', 'AVERAGE_DRIVER_AGE', 'MIN_YEARS_UK_RESIDENCY', 'MAX_YEARS_UK_RESIDENCY', 'AVERAGE_YEARS_UK_RESIDENCY', 'MIN_CREDIT_SCORE', 'MAX_CREDIT_SCORE', 'AVERAGE_CREDIT_SCORE', 'MIN_NRCONVICTIONS_LONGER', 'MAX_NRCONVICTIONS_LONGER', 'AVERAGE_NRCONVICTIONS_LONGER', 'MIN_CLAIMSCOUNT_LONGER', 'MAX_CLAIMSCOUNT_LONGER', 'AVERAGE_CLAIMSCOUNT_LONGER', 'TOP5PREMIUM']

Date Columns:
['VEHICLE_DATEMANUFACTURED', 'VEHICLE_DATEFIRSTREGD', 'VEHICLE_PURCHASEDATE']

Categorical Columns:
['MARITALSTATUS', 'NOOFCHILDREN', 'HOMEOWNERIND', 'VEHICLE_LEFTORRIGHTHANDDRIVE', 'DRIVER1_LICENCETYPE', 'DRIVER1_OTHERV

/var/folders/s6/qrs3nd7546gbp71pw5fqrnzr0000gn/T/ipykernel_13961/661428004.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df[column])
/var/folders/s6/qrs3nd7546gbp71pw5fqrnzr0000gn/T/ipykernel_13961/661428004.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df[column])
/var/folders/s6/qrs3nd7546gbp71pw5fqrnzr0000gn/T/ipykernel_13961/661428004.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df[column])
/var/folders/s6/qrs3nd7546gbp71pw5fqrnzr0000gn/T/ipykernel_13961/661428004.py:23: UserWarning: Could not infer form

In [24]:
len(numerical_columns)+len(date_columns)+len(categorical_columns)+len(text_columns)

59

In [25]:
# Display categorized columns and their types
print("Numerical Columns, their Types and Unique Values:")
for col in numerical_columns:
    print(f"{col}: {df[col].dtype}, {df[col].nunique()}")

print("\nDate Columns, their Types and Unique Values:")
for col in date_columns:
    print(f"{col}: {df[col].dtype}, {df[col].nunique()}")

print("\nCategorical Columns, their Types and Unique Values:")
for col in categorical_columns:
    print(f"{col}: {df[col].dtype}, {df[col].nunique()}")

print("\nText Columns, their Types and Unique Values:")
for col in text_columns:
    print(f"{col}: {df[col].dtype}, {df[col].nunique()}")

Numerical Columns, their Types and Unique Values:
DAYSTOINCEPTION: float64, 30
DRIVER1_AGE: float64, 66
DRIVER1_NOOFYEARSUKRESIDENCY: float64, 83
NOOFVEHICLESAVAILABLETOFAMILY: float64, 9
YEARS_NOCLAIMSDISCOUNT: float64, 10
VEHICLE_VALUE: float64, 5735
VEHICLE_ANNUALMILEAGE: float64, 611
VEHICLE_AGE: float64, 21
VEHICLE_NOOFSEATS: float64, 8
DRIVER1_CREDITSCORE: float64, 18
VEHICLE_CUBICCAPACITY: float64, 394
DRIVER1_NRCONVICTIONS_LONGER: float64, 4
DRIVER1_CLAIMSCOUNT_LONGER: float64, 5
UNDISCLOSED_CLAIMS: float64, 2
NUMBER_OF_DRIVERS: float64, 4
MIN_DRIVER_AGE: float64, 66
MAX_DRIVER_AGE: float64, 66
AVERAGE_DRIVER_AGE: float64, 307
MIN_YEARS_UK_RESIDENCY: float64, 83
MAX_YEARS_UK_RESIDENCY: float64, 83
AVERAGE_YEARS_UK_RESIDENCY: float64, 388
MIN_CREDIT_SCORE: float64, 18
MAX_CREDIT_SCORE: float64, 18
AVERAGE_CREDIT_SCORE: float64, 172
MIN_NRCONVICTIONS_LONGER: float64, 4
MAX_NRCONVICTIONS_LONGER: float64, 4
AVERAGE_NRCONVICTIONS_LONGER: float64, 10
MIN_CLAIMSCOUNT_LONGER: float64, 

In [26]:
# 2. Handling Date Columns

current_date = pd.to_datetime('2024-07-01')
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)

# Calculate the age of the vehicle at the specified current date
df['TIME_SINCE_MANUF'] = (current_date - df['VEHICLE_DATEMANUFACTURED']).dt.days / 365.25
# Calculate the time since the vehicle was first registered at the specified current date
df['TIME_SINCE_FIRST_REGD'] = (current_date - df['VEHICLE_DATEFIRSTREGD']).dt.days / 365.25
# Calculate the time since the vehicle was purchased at the specified current date
df['TIME_SINCE_PURCHASE'] = (current_date - df['VEHICLE_PURCHASEDATE']).dt.days / 365.25

# Delete the original date columns
columns_to_delete = [
    'VEHICLE_DATEMANUFACTURED', 
    'VEHICLE_DATEFIRSTREGD', 
    'VEHICLE_PURCHASEDATE'
]
df.drop(columns=columns_to_delete, inplace=True)

date_columns = [
    'TIME_SINCE_MANUF',
    'TIME_SINCE_FIRST_REGD',
    'TIME_SINCE_PURCHASE'
]

# Move column TOP5PREMIUM to the last position of dataframe
df = df[[col for col in df if col != 'TOP5PREMIUM'] + ['TOP5PREMIUM']]

# Display the first few rows to verify the conversion
print(df[date_columns].head())

   TIME_SINCE_MANUF  TIME_SINCE_FIRST_REGD  TIME_SINCE_PURCHASE
0          9.497604               9.497604             0.646133
1         10.496920              10.496920             5.831622
2          9.497604               9.497604             0.646133
3         15.496235              15.496235             8.334018
4          2.496920               2.496920             1.664613


In [27]:
# Ensure 'TOP5PREMIUM' is not in the categorical or text columns list
if 'TOP5PREMIUM' in categorical_columns:
    categorical_columns.remove('TOP5PREMIUM')
if 'TOP5PREMIUM' in text_columns:
    text_columns.remove('TOP5PREMIUM')

# 3. Encoding Categorical Columns with LabelEncoder
label_encoders = {}
label_mappings = {}

# Process each categorical column with LabelEncoder
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le
    # Store the mapping of original values to encoded values
    label_mappings[col] = dict(zip(le.classes_, le.transform(le.classes_)))

# Display the label mappings
print("Label Encodings:")
for col, mapping in label_mappings.items():
    print(f"Mapping for column '{col}':")
    print(mapping)
    print()

# 4. Encoding Text Columns with Target Encoding
target = df['TOP5PREMIUM']
target_encoder = TargetEncoder()

# Apply target encoding to each text column
for col in text_columns:
    df[col] = target_encoder.fit_transform(df[col], target)


Label Encodings:
Mapping for column 'MARITALSTATUS':
{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'M': 4, 'S': 5, 'W': 6}

Mapping for column 'NOOFCHILDREN':
{0.0: 0, 1.0: 1}

Mapping for column 'HOMEOWNERIND':
{'N': 0, 'Y': 1}

Mapping for column 'VEHICLE_LEFTORRIGHTHANDDRIVE':
{'L': 0, 'R': 1}

Mapping for column 'DRIVER1_LICENCETYPE':
{'E': 0, 'F': 1, 'H': 2, 'N': 3, 'P': 4}

Mapping for column 'DRIVER1_OTHERVEHICLEOWNEDIND':
{'N': 0, 'Y': 1}

Mapping for column 'MONTHLYPAYMENTSIND':
{'N': 0, 'Y': 1}

Mapping for column 'VEHICLE_USE_TYPE':
{'Commuting': 0, 'Non Standard Use': 1, 'Proposers, Employers Or Business Partners Business': 2, 'Social, Domestic And Pleasure': 3, 'Spouses Business': 4}

Mapping for column 'VEHICLEOVERNIGHTLOCATION':
{'Garaged': 0, 'Kept On Private Property': 1, 'Kept On Public Road': 2, 'Locked Compound': 3, 'Office/Factory Car Park': 4, 'Open Public Car Park': 5, 'Parked On Drive': 6, 'Street (Away From Home)': 7, 'Unlocked Garage': 8}

Mapping for column 'VEHICLE_OWNER

In [28]:
# Display the first few rows to verify the encoding
display_top_5_rows(df, block_size=10)


Columns 1 to 10:
   DAYSTOINCEPTION  MARITALSTATUS  NOOFCHILDREN  HOMEOWNERIND  \
0              2.0              5             1             1   
1             21.0              5             0             0   
2             29.0              5             0             0   
3             10.0              4             1             1   
4              1.0              5             0             1   

   VEHICLE_LEFTORRIGHTHANDDRIVE  DRIVER1_AGE  DRIVER1_LICENCETYPE  \
0                             1         21.0                    1   
1                             1         22.0                    1   
2                             1         20.0                    1   
3                             1         51.0                    1   
4                             1         25.0                    1   

   DRIVER2_LICENCETYPE  DRIVER3_LICENCETYPE  DRIVER4_LICENCETYPE  
0                    1                    5                    5  
1                    1                    

In [29]:
display_missing_values(df)


Columns 1 to 10:
DAYSTOINCEPTION                 0
MARITALSTATUS                   0
NOOFCHILDREN                    0
HOMEOWNERIND                    0
VEHICLE_LEFTORRIGHTHANDDRIVE    0
DRIVER1_AGE                     0
DRIVER1_LICENCETYPE             0
DRIVER2_LICENCETYPE             0
DRIVER3_LICENCETYPE             0
DRIVER4_LICENCETYPE             0
dtype: int64

Columns 11 to 20:
DRIVER1_NOOFYEARSUKRESIDENCY     0
DRIVER1_OTHERVEHICLEOWNEDIND     0
DRIVER2_OTHERVEHICLEOWNEDIND     0
DRIVER3_OTHERVEHICLEOWNEDIND     0
DRIVER4_OTHERVEHICLEOWNEDIND     0
POSTCODESECTOR                   0
NOOFVEHICLESAVAILABLETOFAMILY    0
MONTHLYPAYMENTSIND               0
YEARS_NOCLAIMSDISCOUNT           0
VEHICLE_VALUE                    0
dtype: int64

Columns 21 to 30:
VEHICLE_ANNUALMILEAGE       0
VEHICLE_USE_TYPE            0
VEHICLEOVERNIGHTLOCATION    0
VEHICLE_OWNERSHIP           0
VEHICLE_MODIFIEDIND         0
VEHICLE_AGE                 0
VEHICLE_BODYTYPE            0
VEHICLE_FUELTYPE  

In [30]:
null_rows_count = df[
    df['MIN_CREDIT_SCORE'].isnull() & 
    df['MAX_CREDIT_SCORE'].isnull() & 
    df['AVERAGE_CREDIT_SCORE'].isnull()
].shape[0]
print(f"Number of rows where DRIVER3_AGE, DRIVER3_OTHERVEHICLEOWNEDIND, and DRIVER3_CREDITSCORE are all not null: {null_rows_count}")

Number of rows where DRIVER3_AGE, DRIVER3_OTHERVEHICLEOWNEDIND, and DRIVER3_CREDITSCORE are all not null: 835


In [31]:
df_cleaned = df.dropna()

In [32]:
df_cleaned.isnull().sum().sum()

0

In [33]:
display_unique_values(df_cleaned, block_size=10)


Columns 1 to 10:
DAYSTOINCEPTION                 30
MARITALSTATUS                    7
NOOFCHILDREN                     2
HOMEOWNERIND                     2
VEHICLE_LEFTORRIGHTHANDDRIVE     2
DRIVER1_AGE                     66
DRIVER1_LICENCETYPE              5
DRIVER2_LICENCETYPE              6
DRIVER3_LICENCETYPE              6
DRIVER4_LICENCETYPE              6
dtype: int64

Columns 11 to 20:
DRIVER1_NOOFYEARSUKRESIDENCY       83
DRIVER1_OTHERVEHICLEOWNEDIND        2
DRIVER2_OTHERVEHICLEOWNEDIND        3
DRIVER3_OTHERVEHICLEOWNEDIND        3
DRIVER4_OTHERVEHICLEOWNEDIND        3
POSTCODESECTOR                   8508
NOOFVEHICLESAVAILABLETOFAMILY       9
MONTHLYPAYMENTSIND                  2
YEARS_NOCLAIMSDISCOUNT             10
VEHICLE_VALUE                    5722
dtype: int64

Columns 21 to 30:
VEHICLE_ANNUALMILEAGE       609
VEHICLE_USE_TYPE              5
VEHICLEOVERNIGHTLOCATION      9
VEHICLE_OWNERSHIP             8
VEHICLE_MODIFIEDIND           2
VEHICLE_AGE                 

In [34]:
display_top_5_rows(df_cleaned, block_size=10)


Columns 1 to 10:
   DAYSTOINCEPTION  MARITALSTATUS  NOOFCHILDREN  HOMEOWNERIND  \
0              2.0              5             1             1   
1             21.0              5             0             0   
2             29.0              5             0             0   
3             10.0              4             1             1   
4              1.0              5             0             1   

   VEHICLE_LEFTORRIGHTHANDDRIVE  DRIVER1_AGE  DRIVER1_LICENCETYPE  \
0                             1         21.0                    1   
1                             1         22.0                    1   
2                             1         20.0                    1   
3                             1         51.0                    1   
4                             1         25.0                    1   

   DRIVER2_LICENCETYPE  DRIVER3_LICENCETYPE  DRIVER4_LICENCETYPE  
0                    1                    5                    5  
1                    1                    

   AVERAGE_DRIVER_AGE  MIN_YEARS_UK_RESIDENCY  MAX_YEARS_UK_RESIDENCY  \
0           35.000000                     5.0                     7.0   
1           35.500000                    22.0                    49.0   
2           42.666667                    20.0                    57.0   
3           50.000000                    49.0                    51.0   
4           43.000000                    25.0                    53.0   

   AVERAGE_YEARS_UK_RESIDENCY  MIN_CREDIT_SCORE  MAX_CREDIT_SCORE  \
0                    6.000000             275.0             325.0   
1                   35.500000             350.0             375.0   
2                   42.666667             400.0             998.0   
3                   50.000000             450.0             450.0   
4                   43.000000             175.0             350.0   

   AVERAGE_CREDIT_SCORE  INDICATOR_NRCONVICTIONS_RECENT  \
0            300.000000                               0   
1            362.500000     

In [35]:
# Counting unique values in the column 'DRIVER1_NONMOTORINGCONVICTIONIND'
value_counts = df_cleaned['VEHICLE_NOOFSEATS'].value_counts()

# Display the result
print(value_counts)


VEHICLE_NOOFSEATS
5.0    86027
4.0    14076
7.0     3669
2.0     1226
8.0       76
3.0       52
6.0       36
1.0        4
Name: count, dtype: int64


In [36]:
df_cleaned.to_parquet('fe_df_cleaned.parquet', index=False)

In [37]:
fe_cleaned_overall = df_cleaned[(df_cleaned['TOP5PREMIUM'] >= 0) & (df_cleaned['TOP5PREMIUM'] <= 3000)]
fe_cleaned_overall.to_parquet('fe_cleaned_overall.parquet')